In [1]:
# @title Height Encoded

def height_encode(img_data, z_normalized, max_height, z):
    """
    Encodes height information into an image by adjusting pixel values based
    on the z-coordinate data, with max-pooling applied where multiple values
    compete for the same pixel.

    Parameters:
    -----------
    img_data : tuple
        A tuple containing the following elements:
        - file_name: The name of the file.
        - img_size_x: The width of the image.
        - img_size_y: The height of the image.
        - x_norm: A list or array of normalized x-coordinates.
        - y_norm: A list or array of normalized y-coordinates.
        - points: A list or array of points in the image.

    z_normalized : array-like
        An array of normalized z-coordinates to be encoded as pixel values in
        the output image.

    max_height : 2D array
        A 2D array representing the maximum height values for each pixel, used
        to determine which z-coordinate should be encoded when multiple values
        compete for the same pixel.

    z : array-like
        An array of z-coordinates corresponding to the points, used to update
        the max_height array.

    Returns:
    --------
    numpy.ndarray
        An image (3-channel) where the height information is encoded based on
        the provided z-coordinate data.
    """

    file_name, img_size_x, img_size_y, x_norm, y_norm, points = img_data

    HE_image = np.zeros((img_size_y, img_size_x, 3), dtype=np.uint8)

    # Plot points, maxpooling where multiple values compete for a pixel
    for i in range(len(points)):
        if z[i] > max_height[y_norm[i], x_norm[i]]:
            max_height[y_norm[i], x_norm[i]] = z[i]
            HE_image[y_norm[i], x_norm[i]] = [z_normalized[i]] * 3

    return HE_image

In [ ]:
# @title Height Difference Encoded

def height_difference_encode(img_data, min_height, max_height, z):
    """
    Encodes the height difference information into an image by calculating the
    difference between the maximum and minimum z-coordinates for each pixel
    and mapping this difference to pixel values.

    Parameters:
    -----------
    img_data : tuple
        A tuple containing the following elements:
        - file_name: The name of the file.
        - img_size_x: The width of the image.
        - img_size_y: The height of the image.
        - x_norm: A list or array of normalized x-coordinates.
        - y_norm: A list or array of normalized y-coordinates.
        - points: A list or array of points in the image.

    min_height : 2D array
        A 2D array representing the minimum height values for each pixel,
        updated with the minimum z-coordinates encountered.

    max_height : 2D array
        A 2D array representing the maximum height values for each pixel,
        updated with the maximum z-coordinates encountered.

    z : array-like
        An array of z-coordinates corresponding to the points, used to update
        the min_height and max_height arrays.

    Returns:
    --------
    numpy.ndarray
        An image (3-channel) where the height difference information is
        encoded based on the calculated difference between max and min
        z-coordinates.
    """

    file_name, img_size_x, img_size_y, x_norm, y_norm, points = img_data

    HDE_image = np.zeros((img_size_y, img_size_x, 3), dtype=np.uint8)

    # Update data for min/max values of z across the whole image
    for i in range(len(points)):
        if z[i] < min_height[y_norm[i], x_norm[i]]:
            min_height[y_norm[i], x_norm[i]] = z[i]
        if z[i] > max_height[y_norm[i], x_norm[i]]:
            max_height[y_norm[i], x_norm[i]] = z[i]

    # Calc height difference (Z axis), assign zero to empty
    # regions or regions with single height value.
    height_diff = max_height - min_height
    height_diff[height_diff == np.inf] = 0
    height_diff = (height_diff / height_diff.max() * 255).astype(np.uint8)

    for i in range(img_size_y):
        for j in range(img_size_x):
            if height_diff[i, j] > 0:
                HDE_image[i, j] = [height_diff[i, j]] * 3

    return HDE_image


In [ ]:
# @title Point Count Encoded

def point_count_encode(img_data, norm_radius):
    """
    Encodes point density information into an image by counting the number
    of points within each pixel and applying localized normalization based
    on a given radius.

    Parameters:
    -----------
    img_data : tuple
        A tuple containing the following elements:
        - file_name: The name of the file.
        - img_size_x: The width of the image.
        - img_size_y: The height of the image.
        - x_norm: A list or array of normalized x-coordinates.
        - y_norm: A list or array of normalized y-coordinates.
        - points: A list or array of points in the image.

    norm_radius : int
        The radius for applying localized normalization around each pixel,
        used to normalize the point counts.

    Returns:
    --------
    numpy.ndarray
        An image (3-channel) where the point density information is encoded
        based on the normalized count of points within each pixel.
    """

    file_name, img_size_x, img_size_y, x_norm, y_norm, points = img_data

    PCE_image = np.zeros((img_size_y, img_size_x, 3), dtype=np.uint8)

    # Empty matrix to counts the points
    point_count = np.zeros((img_size_y, img_size_x), dtype=np.int64)

    # Count the number of points for each pixel
    for i in range(len(points)):
        point_count[y_norm[i], x_norm[i]] += 1

    # Filter applied for localised normalisation around given radius
    local_max = maximum_filter(point_count, size=norm_radius)

    # Use local maximum to normalise filtered area
    normalized_count = np.zeros_like(point_count)
    for i in range(img_size_y):
        for j in range(img_size_x):
            if local_max[i, j] > 0:
                normalized_count[i, j] = (point_count[i, j] / local_max[i, j]
                                          * 255).astype(np.uint8)

    for i in range(img_size_y):
        for j in range(img_size_x):
            if normalized_count[i, j] > 0:
                PCE_image[i, j] = [normalized_count[i, j]] * 3

    return PCE_image


In [ ]:
# @title Encoding Handler

def encode_images(image_scale = 10,
                  local_normalisation_radius = 128,
                  encode_type = "Height"):
    """
    Processes a directory of point clouds, encodes them into images based on
    the specified encoding type, and saves the resulting images.

    Parameters:
    -----------
    image_scale : int, optional
        A scaling factor applied to the dimensions of the output images,
        calculated based on the range of x and y coordinates in the point
        cloud data. Default is 10.

    local_normalisation_radius : int, optional
        The radius for localized normalization, used in point count encoding
        to normalize point counts within a specified area around each pixel.
        Default is 128.

    encode_type : str, optional
        The type of encoding to apply. Options are "Height", "Height Difference",
        or "Point Count". Default is "Height".

    Returns:
    --------
    tuple
        A tuple containing two lists:
        - image_visual: A list of the encoded images as numpy arrays.
        - image_label: A list of the corresponding file names for each image.
    """

    image_visual, image_label = [], []

    for pointcloud in Path("Pointclouds").iterdir():
        if pointcloud.is_file():

            file_name = pointcloud.name
            pcd = o3d.io.read_point_cloud(f"Pointclouds/{file_name}")
            points = np.array(pcd.points)

            x = points[:, 0]
            y = points[:, 1]
            z = points[:, 2]

            img_size_x = int(image_scale * abs(x.min() - x.max()))
            img_size_y = int(image_scale * abs(y.min() - y.max()))

            x_normalized = ((x - x.min()) / (x.max()
                             - x.min()) * (img_size_x - 1)).astype(np.int64)
            y_normalized = ((y - y.min()) / (y.max()
                             - y.min()) * (img_size_y - 1)).astype(np.int64)

            max_height = np.full((img_size_y, img_size_x), -np.inf)

            image_data = [
                file_name[:-4],
                img_size_x,
                img_size_y,
                x_normalized,
                y_normalized,
                points
            ]

            if encode_type == "Height":
                z_normalized = ((z - z.min()) / (z.max()
                             - z.min()) * 255).astype(np.uint8)

                Encoded_image = height_encode(
                    image_data,
                    z_normalized,
                    max_height,
                    z
                )
            elif encode_type == "Height Difference":

                min_height = np.full((img_size_y, img_size_x), np.inf)

                Encoded_image = height_difference_encode(
                    image_data,
                    min_height,
                    max_height,
                    z
                )
            elif encode_type == "Point Count":
                Encoded_image = point_count_encode(
                    image_data,
                    local_normalisation_radius
                )


            img = Image.fromarray(Encoded_image)
            img.save(
                f"Processed images/{encode_type} Encoded/{image_data[0]}.png"
            )

            image_visual.append(Encoded_image)
            image_label.append(file_name)

    return image_visual, image_label